## Imports

In [14]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets

from utils import combine_strings_with_whitespace

In [15]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/fsx/home-augustas/elk')

In [16]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk']

In [17]:
from templates import DatasetTemplates

In [18]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

# Load the dataset

In [19]:
# Set dataset_dict to test_dataset
dataset_dict = datasets.DatasetDict.load_from_disk("datasets/burns_datasets_VINC_ag_news_ppo_training_raw")
dataset_dict

DatasetDict({
    ag_news: Dataset({
        features: ['text', 'label'],
        num_rows: 6600
    })
})

In [20]:
sum(dataset.num_rows for dataset in dataset_dict.values())

6600

In [21]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4


## Load the templates

In [22]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [23]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15


In [24]:
%%time

# Reproducibility
random.seed(2023)

ALLOWED_KEYS = ["prompt", "best_response", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    # if dataset_name != "ag_news": continue
    # if dataset_name != "imdb": continue
    # if dataset_name != "dbpedia_14": continue
    # if dataset_name != "piqa": continue

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        q, a = template.apply(new_entry)
        new_entry["prompt"] = combine_strings_with_whitespace(a, q)
        if not new_entry["prompt"][-1].isspace():
            new_entry["prompt"] += " "

        # We can now change the label to whether the sample is truthful or not
        new_entry["best_response"] = a.strip()

        # Remove all other keys
        new_entry = { k: v for k, v in new_entry.items() if k in ALLOWED_KEYS }

        # Append to the new dataset
        new_dataset.append(new_entry)

        # print(new_entry["prompt"])
        # print(new_entry["prompt"] + new_entry["best_response"])
        # print("-----------------------------------")
        # if idx >= 10 - 1: break

ag_news


CPU times: user 11.7 s, sys: 0 ns, total: 11.7 s
Wall time: 11.7 s


In [25]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['original_dataset', 'template_name', 'prompt', 'best_response'],
    num_rows: 6600
})

In [26]:
my_dataset[0]

{'original_dataset': 'ag_news',
 'template_name': 'burns_2',
 'prompt': "Consider the following example: ''' New Star-Type Resembles Stillborn Star When a binary star system starts to transfer mass, one of the twins may well win out, leaving its companion to occupy a strange region half way between a star and a planet. A new star-type of this sort has been found, which resembles the infrared ash of a stillborn star. ''' Choice 1: Science and technology. Choice 2: Business.Between choice 1 and choice 2, what is the topic of this example? ",
 'best_response': 'Science and technology'}

In [27]:
Counter(my_dataset["original_dataset"])

Counter({'ag_news': 6600})

In [28]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_ag_news_ppo_training.parquet")

Creating parquet from Arrow format: 100%|██████████| 7/7 [00:00<00:00, 556.88ba/s]


2564363

In [29]:
!ls -lah datasets

total 125K
drwxr-xr-x 4 augustas Domain Users  33K Jun 19 16:14 .
drwxr-xr-x 4 augustas Domain Users  33K Jun 19 15:59 ..
drwxr-xr-x 3 augustas Domain Users  25K Jun 19 16:07 burns_datasets_VINC_ag_news_ppo_training_raw
drwxr-xr-x 3 augustas Domain Users  25K Jun 19 15:59 burns_datasets_VINC_ag_news_test_raw
-rw-r--r-- 1 augustas Domain Users 188K Jun 19 16:00 burns_datasets_VINC_ag_news_train.parquet
-rw-r--r-- 1 augustas Domain Users 187K Jun 19 16:12 burns_datasets_VINC_ag_news_validation.parquet
-rw-r--r-- 1 augustas Domain Users 1.2M Jun 19 16:14 ppo_training_dataset.parquet
